## cross validation

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import pandas as pd
import tensorflow as tf
import theano
import keras 

import numpy as np
import matplotlib.pyplot as plt
from time import time

### data preprocessing

In [4]:
df = pd.read_csv('../Artificial_Neural_Networks/Churn_Modelling.csv')
print df.shape
X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
print X.shape

# remove one dummy variable to avoid dummy variable trap
X = X[:, 1:]
print X.shape

(10000, 14)
(10000, 12)
(10000, 11)


### train test split & feature scaling 

In [5]:
S = lambda *x: [i.shape for i in x]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# print S(X_train,X_test, y_train, y_test)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit on training set
X_train = sc.fit_transform(X_train)
# only transform on test set
X_test = sc.transform(X_test)
print S(X_train,X_test, y_train, y_test)

[(8000, 11), (2000, 11), (8000,), (2000,)]


### build k-fold func ANN
## Dropout and GridSearch (parameter tuning)

In [6]:
from keras.layers import Dropout

from keras.models import Sequential
from keras.layers import Dense

from keras.wrappers.scikit_learn import KerasClassifier
# k-fold cross validation
from sklearn.model_selection import GridSearchCV

In [7]:
def build_classifier(opt, P, h1, h2):
    classifier = Sequential()
    # first hidden layer
    # classifier.add(Dense(units = 6, input_shape=(11,),  kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units = h1, input_dim=11,  kernel_initializer='uniform', activation='relu'))
    # Dropout
    classifier.add(Dropout(p= P))
    
    # second hidden layer
    classifier.add(Dense(units = h2,  kernel_initializer='he_uniform', activation='relu'))
    # Dropout
    classifier.add(Dropout(p= P))    
    
    # ouput layer
    classifier.add(Dense(units = 1,  kernel_initializer='he_uniform', activation='sigmoid'))
    # compiling the ANN
    classifier.compile(optimizer= opt, loss='binary_crossentropy', metrics= ['accuracy'])
#     print classifier.summary()
    return classifier

classifier = KerasClassifier(build_fn= build_classifier)

In [8]:
parameters = {
    'batch_size':[16, 32, 64],
    'nb_epoch':[ 100, 300],
    'opt':['adam'],
    'P':[0.01],
    'h1':[6, 12, 24],
    'h2':[3, 6, 12]
}

In [9]:
grid_sesarch = GridSearchCV(estimator= classifier, param_grid= parameters, 
                           scoring= 'accuracy', cv= 10, n_jobs=-1)

In [ ]:
if __name__ == "__main__":
    import sys
    from time import time
    start = time()
    grid = grid_sesarch.fit(X_train, y_train)
    eclipse = time()-start
    print 'used: {:.2f} s'.format(eclipse)
    
    best_parameters = grid.best_params_
    best_accuracy = grid.best_score_

/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.01)`
  import sys
/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.01)`
  import sys
/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.01)`
  import sys
/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.01)`
  import sys
/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.01)`
  if sys.path[0] == '':
/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
7200/7200 [==============================] - 1s - loss: 0.5130 - acc: 0.7951     
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.5115 - acc: 0.7911     
Epoch 2/10
1056/7200 [===>..........................] - ETA: 0s - loss: 0.4004 - acc: 0.8239
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.4082 - acc: 0.8250     
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.3930 - acc: 0.8356     
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.3953 - acc: 0.8340     
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.3906 - acc: 0.8378     
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.3732 - acc: 0.8507     
Epoch 4/10
7200/7200 [==============================] - 1s - loss: 0.3680 - acc: 0.8536     
Epoch 4/10
7200/7200 [==============================] - 1s - loss: 0.3589 - acc: 0.8531     
Epoch 5/10
7200/7200 [===============

7200/7200 [==============================] - 1s - loss: 0.3664 - acc: 0.8511     
Epoch 9/10
3392/7200 [=============>................] - ETA: 0s - loss: 0.3605 - acc: 0.8564
Epoch 10/10
7200/7200 [==============================] - 1s - loss: 0.3707 - acc: 0.8521     
Epoch 10/10
7200/7200 [==============================] - 1s - loss: 0.3666 - acc: 0.8543     
Epoch 10/10
7200/7200 [==============================] - 1s - loss: 0.4969 - acc: 0.7963     
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.4958 - acc: 0.7994     
Epoch 2/10
 912/7200 [==>...........................] - ETA: 1s - loss: 0.4166 - acc: 0.8015
Epoch 2/10
2864/7200 [==========>...................] - ETA: 0s - loss: 0.4262 - acc: 0.8055
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.4060 - acc: 0.8185     
Epoch 3/10
6256/7200 [=========================>....] - ETA: 0s - loss: 0.4073 - acc: 0.8216
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.4056 - ac

7200/7200 [==============================] - 1s - loss: 0.4103 - acc: 0.8511     
Epoch 6/10
7200/7200 [==============================] - 1s - loss: 0.3990 - acc: 0.8479     
Epoch 7/10
  16/7200 [..............................] - ETA: 1s - loss: 0.2845 - acc: 0.9375Epoch 7/10


Epoch 8/10
7200/7200 [==============================] - 1s - loss: 0.3870 - acc: 0.8504     
Epoch 8/10
7200/7200 [==============================] - 1s - loss: 0.3759 - acc: 0.8574     
Epoch 8/10
6608/7200 [==========================>...] - ETA: 0s - loss: 0.3818 - acc: 0.8526
Epoch 8/10
7200/7200 [==============================] - 1s - loss: 0.3712 - acc: 0.8586     
Epoch 9/10
7200/7200 [==============================] - 1s - loss: 0.3772 - acc: 0.8536     
Epoch 9/10
1024/7200 [===>..........................] - ETA: 1s - loss: 0.3680 - acc: 0.8584
Epoch 10/10
7200/7200 [==============================] - 1s - loss: 0.3719 - acc: 0.8542     
Epoch 10/10
7200/7200 [==============================] - 1s - loss: 

7200/7200 [==============================] - 1s - loss: 0.4195 - acc: 0.8103     
Epoch 5/10
7200/7200 [==============================] - 1s - loss: 0.4203 - acc: 0.8125     
Epoch 5/10
5728/7200 [======================>.......] - ETA: 0s - loss: 0.4189 - acc: 0.8115
Epoch 6/10
7200/7200 [==============================] - 1s - loss: 0.4166 - acc: 0.8150     
Epoch 5/10
2256/7200 [========>.....................] - ETA: 0s - loss: 0.4097 - acc: 0.8320
Epoch 6/10
7200/7200 [==============================] - 1s - loss: 0.4178 - acc: 0.8281     
Epoch 6/10
7200/7200 [==============================] - 1s - loss: 0.4152 - acc: 0.8322     
Epoch 7/10
7200/7200 [==============================] - 1s - loss: 0.4158 - acc: 0.8332     
Epoch 7/10
7200/7200 [==============================] - 1s - loss: 0.4121 - acc: 0.8337     
Epoch 7/10
7200/7200 [==============================] - 1s - loss: 0.4145 - acc: 0.8319     
Epoch 8/10
7200/7200 [==============================] - 1s - loss: 0.4153 - acc: 

...

7200/7200 [==============================] - 1s - loss: 0.3533 - acc: 0.8565     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.3377 - acc: 0.8633     
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.3360 - acc: 0.8608     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3453 - acc: 0.8571     
Epoch 7/10
 352/7200 [>.............................] - ETA: 1s - loss: 0.3739 - acc: 0.8466Epoch 10/10

7200/7200 [==============================] - 1s - loss: 0.3446 - acc: 0.8596     
Epoch 8/10
7200/7200 [==============================] - 2s - loss: 0.4957 - acc: 0.7892     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.3422 - acc: 0.8603     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3881 - acc: 0.8401     
Epoch 3/10
7200/7200 [==============================] - 0s - loss: 0.3414 - acc: 0.8625     
Epoch 10/10
7200/7200 [==============================] - 1s - loss: 0.3592 - acc

7200/7200 [==============================] - 1s - loss: 0.3455 - acc: 0.8581     
Epoch 6/10
7200/7200 [==============================] - 1s - loss: 0.3399 - acc: 0.8610     
Epoch 10/10
7200/7200 [==============================] - 1s - loss: 0.3606 - acc: 0.8574     
Epoch 4/10
1856/7200 [======>.......................] - ETA: 0s - loss: 0.3617 - acc: 0.8491
Epoch 7/10
7200/7200 [==============================] - 1s - loss: 0.3486 - acc: 0.8565     
Epoch 5/10
7200/7200 [==============================] - 1s - loss: 0.3413 - acc: 0.8603     
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.3449 - acc: 0.8597     
Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.3391 - acc: 0.8617     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3429 - acc: 0.8583     
Epoch 7/10
1792/7200 [======>.......................] - ETA: 6s - loss: 0.6237 - acc: 0.7857
Epoch 10/10
7200/7200 [==============================] - 1s - loss: 0.3400 - acc

7200/7200 [==============================] - 0s - loss: 0.4360 - acc: 0.7937     
Epoch 4/10
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.4646 - acc: 0.7944     
Epoch 3/10
7200/7200 [==============================] - 0s - loss: 0.3779 - acc: 0.7975     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.4254 - acc: 0.7937     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.4262 - acc: 0.7944     
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.4170 - acc: 0.7937     
Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.4105 - acc: 0.7944     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.4090 - acc: 0.7937     
Epoch 7/10
7200/7200 [==============================] - 0s - loss: 0.4003 - acc: 0.7944     
Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.4018 - acc: 0.7937     
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.3

7200/7200 [==============================] - 0s - loss: 0.4720 - acc: 0.7956     
Epoch 3/10
7200/7200 [==============================] - 0s - loss: 0.4145 - acc: 0.8240     
Epoch 10/10
7200/7200 [==============================] - 2s - loss: 0.5984 - acc: 0.7847     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.4408 - acc: 0.7956     
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.3756 - acc: 0.8443     
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.4346 - acc: 0.7956     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.4319 - acc: 0.7953     
Epoch 6/10
2112/7200 [=======>......................] - ETA: 0s - loss: 0.4360 - acc: 0.7893
Epoch 10/10
7200/7200 [==============================] - 0s - loss: 0.4275 - acc: 0.7962     
Epoch 7/10
  64/7200 [..............................] - ETA: 0s - loss: 0.3716 - acc: 0.7812
Epoch 5/10
6464/7200 [=========================>....] - ETA: 0s - loss: 0.3944 

7200/7200 [==============================] - 0s - loss: 0.4299 - acc: 0.8194     
Epoch 7/10
Epoch 4/10
7200/7200 [==============================] - 2s - loss: 0.6557 - acc: 0.7357     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.3841 - acc: 0.8278     

Epoch 5/10
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.5153 - acc: 0.7975     
Epoch 3/10
7200/7200 [==============================] - 0s - loss: 0.3882 - acc: 0.8392     
Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.3759 - acc: 0.8367     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.4381 - acc: 0.8142     
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.3680 - acc: 0.8436     
Epoch 7/10
Epoch 10/10
7200/7200 [==============================] - 2s - loss: 0.5903 - acc: 0.7897     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.4183 - acc: 0.8194     
Epoch 5/10
2304/7200 [========>...............

In [ ]:
best_parameters

In [ ]:
best_accuracy

In [ ]:
# GridSearchCV?

In [ ]:
eclipse

In [ ]:
from time import ctime
with open('result_5.txt', 'w+') as fl:
    fl.write(str(best_parameters)+'\n')
    fl.write(str(best_accuracy)+'\n')
    fl.write(str(eclipse)+'\n')
    fl.write(str(ctime())+'\n')

In [ ]:
# if __name__ == "__main__":
#     from time import time
#     start=time()
#     accuracies = cross_val_score(estimator = classifier, X=X_train, y=y_train, cv=10, n_jobs=-1)
#     eclipse = time()-start
#     print 'used: {:.2f} s'.format(eclipse)

In [ ]:
# accuracies

In [ ]:
# accuracies.mean()

In [ ]:
# accuracies.std()

### Predicting a single new observation